In [3]:
!pip install ultralytics

  Using cached ultralytics-8.3.152-py3-none-any.whl.metadata (37 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl.metadata (6.1 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
Using cached ultralytics-8.3.152-py3-none-any.whl (1.0 MB)
Using cached torch-2.7.1-cp312-cp312-win_amd64.whl (216.1 MB)
Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl (1.7 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [1]:
#train, val 8:2
import os
import random
import shutil

def split_dataset(root_dir, train_ratio=0.8, seed=42):
    # 1) 원본 이미지/레이블 디렉터리 경로
    images_dir = os.path.join("C:/Users/jenni/Desktop/final_data1/train", "images")
    labels_dir = os.path.join("C:/Users/jenni/Desktop/final_data1/train", "labels")

    # 2) train/val 이미지 & 레이블을 저장할 디렉터리 경로
    train_images_dir = os.path.join("C:/Users/jenni/Desktop/final_data1", "train1", "images")
    train_labels_dir = os.path.join("C:/Users/jenni/Desktop/final_data1", "train1", "labels")
    val_images_dir = os.path.join("C:/Users/jenni/Desktop/final_data1", "val", "images")
    val_labels_dir = os.path.join("C:/Users/jenni/Desktop/final_data1", "val", "labels")

    train_ratio = 0.8 
    # 3) train/val 디렉터리 생성 (이미 있으면 무시)
    for d in (train_images_dir, train_labels_dir, val_images_dir, val_labels_dir):
        os.makedirs(d, exist_ok=True)

    # 4) images_dir 안에 있는 모든 파일명을 리스트로 가져오기
    #    (폴더 안에 이미지가 .jpg, .png, 기타 다른 확장자로 섞여 있어도 모두 포함)
    all_images = [
        f
        for f in os.listdir(images_dir)
        if os.path.isfile(os.path.join(images_dir, f))
    ]

    if len(all_images) == 0:
        raise RuntimeError(f"이미지 폴더에 파일이 하나도 없습니다: {images_dir}")

    # 5) 랜덤으로 셔플하고 split index 계산
    random.seed(seed)
    random.shuffle(all_images)
    split_idx = int(len(all_images) * train_ratio)

    train_images = all_images[:split_idx]
    val_images = all_images[split_idx:]

    # 6) train 이미지/레이블 복사
    for img_file in train_images:
        # -- 이미지 파일 복사
        src_img = os.path.join(images_dir, img_file)
        dst_img = os.path.join(train_images_dir, img_file)
        shutil.copy2(src_img, dst_img)

        # -- 레이블 파일(.txt) 복사
        base_name, _ = os.path.splitext(img_file)
        label_file = base_name + ".txt"
        src_lbl = os.path.join(labels_dir, label_file)
        dst_lbl = os.path.join(train_labels_dir, label_file)
        if os.path.exists(src_lbl):
            shutil.copy2(src_lbl, dst_lbl)
        else:
            print(f"Warning: train 레이블 파일을 찾을 수 없습니다: {src_lbl}")

    # 7) val 이미지/레이블 복사
    for img_file in val_images:
        src_img = os.path.join(images_dir, img_file)
        dst_img = os.path.join(val_images_dir, img_file)
        shutil.copy2(src_img, dst_img)

        base_name, _ = os.path.splitext(img_file)
        label_file = base_name + ".txt"
        src_lbl = os.path.join(labels_dir, label_file)
        dst_lbl = os.path.join(val_labels_dir, label_file)
        if os.path.exists(src_lbl):
            shutil.copy2(src_lbl, dst_lbl)
        else:
            print(f"Warning: val 레이블 파일을 찾을 수 없습니다: {src_lbl}")

    print(f"총 이미지 수: {len(all_images)}")
    print(f"  → Train: {len(train_images)}장 (비율 {train_ratio*100:.0f}%)")
    print(f"  → Val:   {len(val_images)}장 (비율 {(1-train_ratio)*100:.0f}%)")


if __name__ == "__main__":
    # 실제 root_dir 경로를 본인 PC에 맞게 수정하세요.
    # (예: "C:/Users/USER/Desktop/augmented_data1")
    root_dir = "C:/Users/jenni/Desktop/final_data1"
    split_dataset(root_dir, train_ratio=0.8, seed=42)


총 이미지 수: 3000
  → Train: 2400장 (비율 80%)
  → Val:   600장 (비율 20%)


In [26]:
from ultralytics import YOLO

def train_yolo_for_method(method, base_path="C:／Users／USER／Desktop／augmented1", epochs=50):
    # 1. 각 증강 방식에 맞는 데이터셋 경로 설정
    dataset_path = os.path.join(base_path, method)
    yaml_path = os.path.join(dataset_path, "data.yaml")
    create_data_yaml(dataset_path, yaml_path)

    # 2. 항상 동일한 사전 학습 모델로부터 학습 시작
    model = YOLO("yolov8n-seg.pt")

    # 3. 모든 학습에 동일한 하이퍼파라미터 적용
    model.train(
        data=yaml_path,
        epochs=epochs,       # 에포크 횟수 (50으로 고정)
        imgsz=640,           # 이미지 크기 (640으로 고정)
        name=f"yolov8n_{method}", # 결과 저장 폴더 이름 지정
        project="C:／Users／USER／Desktop", # 결과 저장 최상위 경로
        save=True,           # 모델 체크포인트 저장
        resume=False         # 이어서 학습하지 않고 항상 처음부터 시작
    )


In [ ]:
#조건1 양옆10%만 적용한 것의 학습 (aug2)
import os
import cv2
import json
import glob
from ultralytics import YOLO


BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/augmented_data3" # 증강된 데이터의 최상위 경로 (poisson_harmonized 폴더가 이 아래에 있음)
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_TRAIN_RESULTS3" # YOLO 학습 결과가 저장될 경로


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """
    Labelme JSON 파일들을 YOLO segmentation .txt 파일로 변환합니다.
    Args:
        json_dir (str): Labelme JSON 파일들이 있는 디렉토리 경로.
        output_txt_dir (str): 변환된 YOLO .txt 파일들을 저장할 디렉토리 경로.
        img_w (int): 원본 이미지의 너비.
        img_h (int): 원본 이미지의 높이.
        label_to_id_map (dict): 클래스 이름(label)을 ID(숫자)로 매핑하는 딕셔너리.
    """
    os.makedirs(output_txt_dir, exist_ok=True)
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_filename = os.path.basename(json_path).replace(".json", ".txt")
        txt_path = os.path.join(output_txt_dir, txt_filename)

        yolo_lines = []
        for shape in data['shapes']:
            label_name = shape['label']
            if label_name in label_to_id_map:
                class_id = label_to_id_map[label_name]
                points = shape['points']
                
                normalized_points = []
                for x, y in points:
                    # 좌표를 이미지 크기에 맞춰 정규화
                    normalized_points.append(f"{(x / img_w):.6f}")
                    normalized_points.append(f"{(y / img_h):.6f}")
                
                yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
            else:
                print(f"Warning: Class '{label_name}' not found in label_to_id_map. Skipping shape in {json_path}")

        if yolo_lines:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(yolo_lines))
        # else:
        #    # 객체가 없는 이미지도 빈 .txt 파일로 저장하는 것이 YOLO 학습에 더 안전합니다.
        #    with open(txt_path, 'w') as f:
        #        pass # 빈 파일 생성


def create_data_yaml(dataset_base_path, yaml_output_path, class_names):
    """
    YOLOv8 학습을 위한 data.yaml 파일을 생성합니다.
    Args:
        dataset_base_path (str): 증강된 이미지와 라벨이 있는 상위 디렉토리 (예: 'C:/Users/USER/Desktop/augmented_data1/poisson_harmonized')
        yaml_output_path (str): 생성될 data.yaml 파일의 전체 경로.
        class_names (list): 데이터셋의 모든 클래스 이름 목록.
    """
    images_path = os.path.join(dataset_base_path, "images")
    labels_path = os.path.join(dataset_base_path, "labels_yolo") # YOLO 형식의 라벨 폴더

    # 경로를 상대 경로가 아닌 절대 경로로 지정하여 혼란 방지
    train_img_path = images_path
    val_img_path = images_path # 검증 데이터가 따로 없으면 train과 동일하게 설정

    nc = len(class_names)
    names = class_names

    yaml_content = f"""
path: {dataset_base_path}
train: {os.path.relpath(train_img_path, dataset_base_path)} # path 기준으로 상대 경로
val: {os.path.relpath(val_img_path, dataset_base_path)}

nc: {nc}
names: {names}
"""
    # 실제 YOLO 라벨 경로를 data.yaml이 참조하도록 labels 폴더의 이름을 변경하거나 (labels -> labels_yolo),
    # data.yaml 내부에서 라벨 경로를 명시적으로 지정해야 합니다.
    # 일반적으로 YOLO는 'images' 폴더와 같은 레벨의 'labels' 폴더를 기대합니다.
    # 그래서 'labels_yolo'라는 폴더명을 가정했습니다.

    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' file created successfully.")


def train_yolo_for_method(method, epochs=50):
    """
    특정 증강 방식에 대해 YOLOv8 모델을 학습시키고, 검증 결과를 도출합니다.
    Args:
        method (str): 증강 방식 이름 (예: 'poisson_harmonized').
        epochs (int): 학습할 에포크 수.
    """
    print(f"\n--- Starting YOLOv8 training and validation for method: {method} ---")

    # 1. 각 증강 방식에 맞는 데이터셋 경로 설정
    dataset_path = os.path.join(BASE_AUGMENTED_PATH, method)
    images_path = os.path.join(dataset_path, "images")
    labels_json_path = os.path.join(dataset_path, "labels") # Labelme JSON 라벨이 있는 곳
    labels_yolo_path = os.path.join(dataset_path, "labels_yolo") # YOLO .txt 라벨을 저장할 곳

    # 2. 클래스 정보를 미리 정의
    label2id = {}
    id2label = {}
    try:
        with open("classes.txt", "r", encoding="utf-8") as f:
            class_names_from_file = [line.strip() for line in f if line.strip()]
        for i, name in enumerate(class_names_from_file):
            label2id[name] = i
            id2label[i] = name
        print(f"Loaded classes: {id2label}")
    except FileNotFoundError:
        print("Error: 'classes.txt' not found. Please ensure it's in the same directory or provide class info manually.")
        return
    except Exception as e:
        print(f"Error loading classes.txt: {e}")
        return

    # 이미지 크기를 추정하기 위해 첫 번째 이미지 파일을 읽습니다.
    first_image_path = None
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))
    if image_files:
        first_image_path = image_files[0]
        img_temp = cv2.imread(first_image_path)
        if img_temp is None:
            print(f"Error: Could not read image {first_image_path}. Check image file integrity.")
            return
        img_h, img_w = img_temp.shape[:2]
        print(f"Detected image dimensions: W={img_w}, H={img_h}")
    else:
        print(f"Error: No images found in {images_path}. Cannot determine image dimensions for YOLO conversion.")
        return

    # 3. Labelme JSON 라벨을 YOLO Segmentation .txt 형식으로 변환
    print(f"Converting Labelme JSON to YOLO segmentation .txt for {method}...")
    convert_labelme_to_yolo_segmentation(labels_json_path, labels_yolo_path, img_w, img_h, label2id)
    print("Conversion complete.")

    # 4. data.yaml 파일 생성
    yaml_path = os.path.join(dataset_path, "data.yaml")
    create_data_yaml(dataset_path, yaml_path, list(id2label.values()))

    # 5. 항상 동일한 사전 학습 모델로부터 학습 시작
    model = YOLO("yolov8n-seg.pt")
    print(f"Loaded YOLOv8n-seg model.")

    # 6. 모든 학습에 동일한 하이퍼파라미터 적용
    try:
        # 학습 수행
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{method}",
            project=PROJECT_OUTPUT_PATH, # 결과 저장 최상위 경로
            save=True,
            resume=False
        )
        print(f"--- Training for method '{method}' completed. Results saved to {PROJECT_OUTPUT_PATH}/yolov8n_{method} ---")

        # 7. 학습 완료 후 검증 수행 (자동으로 수행되지만 명시적으로 호출 가능)
        # train 시에 이미 val이 포함되므로, 별도로 model.val()을 호출하지 않아도 결과는 results 객체에 포함됩니다.
        # 하지만 특정 체크포인트를 사용하여 검증하고 싶다면 model.val()을 사용할 수 있습니다.
        # 여기서는 학습 결과 객체에서 직접 metrics를 추출하는 방법을 보여줍니다.
        
        # results 객체에서 metrics 추출
        # YOLOv8의 results 객체는 학습 결과 및 검증 지표를 포함합니다.
        # 정확한 키 이름은 YOLOv8 버전에 따라 약간 다를 수 있으니,
        # results 객체의 내용을 확인하는 것이 좋습니다 (예: print(results.metrics)).
        # 일반적으로 segment 모델의 경우, 다음과 같은 키를 가집니다:
        # metrics['metrics/precision(B)'] (Precision)
        # metrics['metrics/recall(B)'] (Recall)
        # metrics['metrics/mAP50(B)'] (mAP50)
        # metrics['metrics/mAP50-95(B)'] (mAP50-95)
        
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Validation Metrics ---")
            
            # Segmentation 모델의 경우 mAP는 'mAP50', 'mAP50-95' 형태로 접근합니다.
            # Precision, Recall, F1-score는 classification 또는 detection에서 더 흔하게 사용되지만,
            # segment 결과에서도 'metrics/precision(B)', 'metrics/recall(B)'와 같은 키로 접근 가능합니다.
            # F1-score는 Precision과 Recall로부터 계산해야 합니다.
            
            precision = results.metrics.get('metrics/precision(B)', 'N/A')
            recall = results.metrics.get('metrics/recall(B)', 'N/A')
            mAP50 = results.metrics.get('metrics/mAP50(B)', 'N/A')
            mAP = results.metrics.get('metrics/mAP50-95(B)', 'N/A') # mAP (0.5 to 0.95 IOU)

            # F1-score 계산
            f1_score = 'N/A'
            if precision != 'N/A' and recall != 'N/A' and (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)

            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1-Score: {f1_score:.4f}")
            print(f"mAP50: {mAP50:.4f}")
            print(f"mAP (50-95): {mAP:.4f}")
            print("------------------------")
        else:
            print("Could not retrieve validation metrics from training results.")

    except Exception as e:
        print(f"Error during YOLO training for method '{method}': {e}")


# --- 사용 예시 ---
if __name__ == "__main__":
    # 학습을 시작할 증강 방식 이름 지정 (예: 이전 증강 스크립트에서 사용한 blending mode와 동일하게)
    target_method = "train" # ⚠️ 여기에 실제로 사용한 증강 방식 폴더 이름을 입력하세요.

    train_yolo_for_method(target_method, epochs=30)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training and validation for method: train ---
Loaded classes: {0: 'ac', 1: 'lc', 2: 'ph', 3: 'pc', 4: 'tc'}
Detected image dimensions: W=1920, H=648
Converting Labelme JSON to YOLO segmentation .txt for train...
Conversion complete.
'C:/Users/jenni/Desktop/augmented_data3\train\data.yaml' file created successfully.
Loaded YOLOv8n-seg model.
Ultralytics 8.3.152  Python-3.11.11 torch-2.7.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/jenni/Desktop/augmented_data3\train\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=F

train: Scanning C:\Users\jenni\Desktop\augmented_data3\train\labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 23.56.5 MB/s, size: 321.6 KB)



c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\jenni\Desktop\augmented_data3\train\labels.cache... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS3\yolov8n_train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS3\yolov8n_train2
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   6%|▋         | 2/32 [00:04<01:01,  2.05s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:52<00:00,  1.64s/it]


                   all       1000       6019      0.694      0.477      0.515      0.367      0.694      0.447      0.461      0.209
                    ac        758       1590      0.816      0.889      0.913      0.717      0.798      0.847       0.84      0.421
                    lc        612       1165      0.737      0.788      0.806       0.62      0.692      0.712      0.685      0.262
                    ph        739       1299      0.671      0.329      0.371      0.207      0.669      0.309      0.342      0.152
                    pc        511        772      0.696      0.267      0.296      0.208      0.749      0.275      0.301      0.168
                    tc        714       1193      0.548       0.11      0.191     0.0822      0.562      0.093      0.138     0.0425
Speed: 0.7ms preprocess, 30.2ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS3\yolov8n_train2
--- Training for method 'train' completed. R

In [ ]:
#조건2 클래스 근처 20%만만 적용한 것+ 원본 파일 학습 (aug2)
import os
import cv2
import json
import glob
from ultralytics import YOLO


BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/augmented_data4" # 증강된 데이터의 최상위 경로 (poisson_harmonized 폴더가 이 아래에 있음)
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_TRAIN_RESULTS4" # YOLO 학습 결과가 저장될 경로


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """
    Labelme JSON 파일들을 YOLO segmentation .txt 파일로 변환합니다.
    Args:
        json_dir (str): Labelme JSON 파일들이 있는 디렉토리 경로.
        output_txt_dir (str): 변환된 YOLO .txt 파일들을 저장할 디렉토리 경로.
        img_w (int): 원본 이미지의 너비.
        img_h (int): 원본 이미지의 높이.
        label_to_id_map (dict): 클래스 이름(label)을 ID(숫자)로 매핑하는 딕셔너리.
    """
    os.makedirs(output_txt_dir, exist_ok=True)
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_filename = os.path.basename(json_path).replace(".json", ".txt")
        txt_path = os.path.join(output_txt_dir, txt_filename)

        yolo_lines = []
        for shape in data['shapes']:
            label_name = shape['label']
            if label_name in label_to_id_map:
                class_id = label_to_id_map[label_name]
                points = shape['points']
                
                normalized_points = []
                for x, y in points:
                    # 좌표를 이미지 크기에 맞춰 정규화
                    normalized_points.append(f"{(x / img_w):.6f}")
                    normalized_points.append(f"{(y / img_h):.6f}")
                
                yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
            else:
                print(f"Warning: Class '{label_name}' not found in label_to_id_map. Skipping shape in {json_path}")

        if yolo_lines:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(yolo_lines))
        # else:
        #    # 객체가 없는 이미지도 빈 .txt 파일로 저장하는 것이 YOLO 학습에 더 안전합니다.
        #    with open(txt_path, 'w') as f:
        #        pass # 빈 파일 생성


def create_data_yaml(dataset_base_path, yaml_output_path, class_names):
    """
    YOLOv8 학습을 위한 data.yaml 파일을 생성합니다.
    Args:
        dataset_base_path (str): 증강된 이미지와 라벨이 있는 상위 디렉토리 (예: 'C:/Users/USER/Desktop/augmented_data1/poisson_harmonized')
        yaml_output_path (str): 생성될 data.yaml 파일의 전체 경로.
        class_names (list): 데이터셋의 모든 클래스 이름 목록.
    """
    images_path = os.path.join(dataset_base_path, "images")
    labels_path = os.path.join(dataset_base_path, "labels_yolo") # YOLO 형식의 라벨 폴더

    # 경로를 상대 경로가 아닌 절대 경로로 지정하여 혼란 방지
    train_img_path = images_path
    val_img_path = images_path # 검증 데이터가 따로 없으면 train과 동일하게 설정

    nc = len(class_names)
    names = class_names

    yaml_content = f"""
path: {dataset_base_path}
train: {os.path.relpath(train_img_path, dataset_base_path)} # path 기준으로 상대 경로
val: {os.path.relpath(val_img_path, dataset_base_path)}

nc: {nc}
names: {names}
"""
    # 실제 YOLO 라벨 경로를 data.yaml이 참조하도록 labels 폴더의 이름을 변경하거나 (labels -> labels_yolo),
    # data.yaml 내부에서 라벨 경로를 명시적으로 지정해야 합니다.
    # 일반적으로 YOLO는 'images' 폴더와 같은 레벨의 'labels' 폴더를 기대합니다.
    # 그래서 'labels_yolo'라는 폴더명을 가정했습니다.

    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' file created successfully.")


def train_yolo_for_method(method, epochs=50):
    """
    특정 증강 방식에 대해 YOLOv8 모델을 학습시키고, 검증 결과를 도출합니다.
    Args:
        method (str): 증강 방식 이름 (예: 'poisson_harmonized').
        epochs (int): 학습할 에포크 수.
    """
    print(f"\n--- Starting YOLOv8 training and validation for method: {method} ---")

    # 1. 각 증강 방식에 맞는 데이터셋 경로 설정
    dataset_path = os.path.join(BASE_AUGMENTED_PATH, method)
    images_path = os.path.join(dataset_path, "images")
    labels_json_path = os.path.join(dataset_path, "labels") # Labelme JSON 라벨이 있는 곳
    labels_yolo_path = os.path.join(dataset_path, "labels_yolo") # YOLO .txt 라벨을 저장할 곳

    # 2. 클래스 정보를 미리 정의
    label2id = {}
    id2label = {}
    try:
        with open("classes.txt", "r", encoding="utf-8") as f:
            class_names_from_file = [line.strip() for line in f if line.strip()]
        for i, name in enumerate(class_names_from_file):
            label2id[name] = i
            id2label[i] = name
        print(f"Loaded classes: {id2label}")
    except FileNotFoundError:
        print("Error: 'classes.txt' not found. Please ensure it's in the same directory or provide class info manually.")
        return
    except Exception as e:
        print(f"Error loading classes.txt: {e}")
        return

    # 이미지 크기를 추정하기 위해 첫 번째 이미지 파일을 읽습니다.
    first_image_path = None
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))
    if image_files:
        first_image_path = image_files[0]
        img_temp = cv2.imread(first_image_path)
        if img_temp is None:
            print(f"Error: Could not read image {first_image_path}. Check image file integrity.")
            return
        img_h, img_w = img_temp.shape[:2]
        print(f"Detected image dimensions: W={img_w}, H={img_h}")
    else:
        print(f"Error: No images found in {images_path}. Cannot determine image dimensions for YOLO conversion.")
        return

    # 3. Labelme JSON 라벨을 YOLO Segmentation .txt 형식으로 변환
    print(f"Converting Labelme JSON to YOLO segmentation .txt for {method}...")
    convert_labelme_to_yolo_segmentation(labels_json_path, labels_yolo_path, img_w, img_h, label2id)
    print("Conversion complete.")

    # 4. data.yaml 파일 생성
    yaml_path = os.path.join(dataset_path, "data.yaml")
    create_data_yaml(dataset_path, yaml_path, list(id2label.values()))

    # 5. 항상 동일한 사전 학습 모델로부터 학습 시작
    model = YOLO("yolov8n-seg.pt")
    print(f"Loaded YOLOv8n-seg model.")

    # 6. 모든 학습에 동일한 하이퍼파라미터 적용
    try:
        # 학습 수행
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{method}",
            project=PROJECT_OUTPUT_PATH, # 결과 저장 최상위 경로
            save=True,
            resume=False
        )
        print(f"--- Training for method '{method}' completed. Results saved to {PROJECT_OUTPUT_PATH}/yolov8n_{method} ---")

        # 7. 학습 완료 후 검증 수행 (자동으로 수행되지만 명시적으로 호출 가능)
        # train 시에 이미 val이 포함되므로, 별도로 model.val()을 호출하지 않아도 결과는 results 객체에 포함됩니다.
        # 하지만 특정 체크포인트를 사용하여 검증하고 싶다면 model.val()을 사용할 수 있습니다.
        # 여기서는 학습 결과 객체에서 직접 metrics를 추출하는 방법을 보여줍니다.
        
        # results 객체에서 metrics 추출
        # YOLOv8의 results 객체는 학습 결과 및 검증 지표를 포함합니다.
        # 정확한 키 이름은 YOLOv8 버전에 따라 약간 다를 수 있으니,
        # results 객체의 내용을 확인하는 것이 좋습니다 (예: print(results.metrics)).
        # 일반적으로 segment 모델의 경우, 다음과 같은 키를 가집니다:
        # metrics['metrics/precision(B)'] (Precision)
        # metrics['metrics/recall(B)'] (Recall)
        # metrics['metrics/mAP50(B)'] (mAP50)
        # metrics['metrics/mAP50-95(B)'] (mAP50-95)
        
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Validation Metrics ---")
            
            # Segmentation 모델의 경우 mAP는 'mAP50', 'mAP50-95' 형태로 접근합니다.
            # Precision, Recall, F1-score는 classification 또는 detection에서 더 흔하게 사용되지만,
            # segment 결과에서도 'metrics/precision(B)', 'metrics/recall(B)'와 같은 키로 접근 가능합니다.
            # F1-score는 Precision과 Recall로부터 계산해야 합니다.
            
            precision = results.metrics.get('metrics/precision(B)', 'N/A')
            recall = results.metrics.get('metrics/recall(B)', 'N/A')
            mAP50 = results.metrics.get('metrics/mAP50(B)', 'N/A')
            mAP = results.metrics.get('metrics/mAP50-95(B)', 'N/A') # mAP (0.5 to 0.95 IOU)

            # F1-score 계산
            f1_score = 'N/A'
            if precision != 'N/A' and recall != 'N/A' and (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)

            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1-Score: {f1_score:.4f}")
            print(f"mAP50: {mAP50:.4f}")
            print(f"mAP (50-95): {mAP:.4f}")
            print("------------------------")
        else:
            print("Could not retrieve validation metrics from training results.")

    except Exception as e:
        print(f"Error during YOLO training for method '{method}': {e}")


# --- 사용 예시 ---
if __name__ == "__main__":
    # 학습을 시작할 증강 방식 이름 지정 (예: 이전 증강 스크립트에서 사용한 blending mode와 동일하게)
    target_method = "train" # ⚠️ 여기에 실제로 사용한 증강 방식 폴더 이름을 입력하세요.

    train_yolo_for_method(target_method, epochs=50)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training and validation for method: train ---
Loaded classes: {0: 'ac', 1: 'lc', 2: 'ph', 3: 'pc', 4: 'tc'}
Detected image dimensions: W=1920, H=648
Converting Labelme JSON to YOLO segmentation .txt for train...
Conversion complete.
'C:/Users/jenni/Desktop/augmented_data4\train\data.yaml' file created successfully.
Loaded YOLOv8n-seg model.
Ultralytics 8.3.152  Python-3.11.11 torch-2.7.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/jenni/Desktop/augmented_data4\train\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=F

train: Scanning C:\Users\jenni\Desktop\augmented_data4\train\labels... 1032 images, 862 backgrounds, 0 corrupt: 100%|██████████| 1894/1894 [00:04<00:00, 425.49it/s]


train: New cache created: C:\Users\jenni\Desktop\augmented_data4\train\labels.cache


c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


val: Fast image access  (ping: 0.10.0 ms, read: 1098.0452.9 MB/s, size: 280.5 KB)


...

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   3%|▎         | 2/60 [00:02<01:27,  1.51s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 60/60 [01:23<00:00,  1.40s/it]


                   all       1894       6232      0.537      0.462      0.453      0.327      0.526      0.433      0.409       0.19
                    ac        767       1641      0.636      0.809      0.771      0.633      0.628      0.775      0.728      0.385
                    lc        666       1214      0.602      0.699      0.685       0.54      0.576      0.648      0.606      0.242
                    ph        771       1372      0.592      0.379      0.388      0.231      0.591      0.363      0.366      0.173
                    pc        528        808      0.454      0.233      0.215      0.145      0.486      0.233       0.21      0.106
                    tc        709       1197      0.401      0.188      0.205     0.0883      0.349      0.145      0.134     0.0449
Speed: 0.7ms preprocess, 29.7ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS4\yolov8n_train
--- Training for method 'train' completed. Re

In [9]:
# -*- copy-paste만 적용
"""YOLOv8 학습 및 검증 스크립트 (Train/Val 분리)

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1GTwD00wWTckTswWBJxy8ye1o3Mllj-3w
"""

import os
import cv2
import json
import glob
from ultralytics import YOLO

# --- 경로 설정 ---
# 데이터셋의 상위 경로 (예: 'augmented_data2' 폴더가 있는 위치)
BASE_DATASET_PATH = "C:/Users/jenni/Desktop/augmented_data2"
# YOLO 학습 결과가 저장될 경로
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_TRAIN_RESULTS2"


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """
    Labelme JSON 파일들을 YOLO segmentation .txt 파일로 변환합니다.
    Args:
        json_dir (str): Labelme JSON 파일들이 있는 디렉토리 경로.
        output_txt_dir (str): 변환된 YOLO .txt 파일들을 저장할 디렉토리 경로.
        img_w (int): 원본 이미지의 너비.
        img_h (int): 원본 이미지의 높이.
        label_to_id_map (dict): 클래스 이름(label)을 ID(숫자)로 매핑하는 딕셔너리.
    """
    os.makedirs(output_txt_dir, exist_ok=True)
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    if not json_files:
        print(f"Warning: No JSON files found in {json_dir}. Skipping conversion for this directory.")
        return

    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_filename = os.path.basename(json_path).replace(".json", ".txt")
        txt_path = os.path.join(output_txt_dir, txt_filename)

        yolo_lines = []
        for shape in data.get('shapes', []):
            label_name = shape.get('label')
            if label_name in label_to_id_map:
                class_id = label_to_id_map[label_name]
                points = shape.get('points', [])

                normalized_points = []
                for x, y in points:
                    # 좌표를 이미지 크기에 맞춰 정규화
                    normalized_points.append(f"{(x / img_w):.6f}")
                    normalized_points.append(f"{(y / img_h):.6f}")

                if normalized_points:
                    yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
            else:
                print(f"Warning: Class '{label_name}' not found in label_to_id_map. Skipping shape in {json_path}")

        # 객체가 있든 없든 항상 .txt 파일을 생성하여 학습 안정성을 높입니다.
        with open(txt_path, 'w', encoding='utf-8') as f:
            if yolo_lines:
                f.write('\n'.join(yolo_lines))
            else:
                pass # 객체가 없는 경우 빈 파일 생성


def create_data_yaml(dataset_base_path, yaml_output_path, class_names):
    """
    YOLOv8 학습을 위한 data.yaml 파일을 생성합니다. (train/val 분리 구조)
    Args:
        dataset_base_path (str): 'train', 'val' 폴더가 있는 상위 디렉토리.
        yaml_output_path (str): 생성될 data.yaml 파일의 전체 경로.
        class_names (list): 데이터셋의 모든 클래스 이름 목록.
    """
    # YOLOv8은 data.yaml에 지정된 'path'를 기준으로 train/val 경로를 찾습니다.
    # 경로는 호환성을 위해 슬래시(/)로 변환합니다.
    path_str = dataset_base_path.replace(os.sep, '/')
    train_images_rel_path = 'train/images'
    val_images_rel_path = 'val/images'

    nc = len(class_names)

    yaml_content = f"""
# YOLOv8 Dataset Configuration File
# Paths are relative to the 'path' directory.

path: {path_str}
train: {train_images_rel_path}
val: {val_images_rel_path}

# Number of classes and names
nc: {nc}
names: {class_names}
"""
    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' file created successfully.")


def run_yolo_training(dataset_name, epochs=50):
    """
    'train'/'val' 구조의 데이터셋으로 YOLOv8 모델을 학습하고 검증합니다.
    Args:
        dataset_name (str): 'train', 'val' 폴더를 포함하는 데이터셋 폴더 이름.
                              (BASE_DATASET_PATH 아래에 위치해야 함)
        epochs (int): 학습할 에포크 수.
    """
    print(f"\n--- Starting YOLOv8 training for dataset: {dataset_name} ---")

    # 1. 경로 설정
    dataset_path = os.path.join(BASE_DATASET_PATH, dataset_name)
    train_path = os.path.join(dataset_path, "train")
    val_path = os.path.join(dataset_path, "val")

    # 필요한 폴더 존재 여부 확인
    if not (os.path.isdir(train_path) and os.path.isdir(val_path)):
        print(f"Error: 'train' and/or 'val' directory not found in '{dataset_path}'")
        return

    train_images_path = os.path.join(train_path, "images")
    train_labels_json_path = os.path.join(train_path, "labels")
    train_labels_yolo_path = os.path.join(train_path, "labels_yolo") # 변환된 YOLO 라벨 저장 위치

    val_labels_json_path = os.path.join(val_path, "labels")
    val_labels_yolo_path = os.path.join(val_path, "labels_yolo") # 변환된 YOLO 라벨 저장 위치

    # 2. 클래스 정보 로드
    label2id = {}
    id2label = {}
    try:
        with open("classes.txt", "r", encoding="utf-8") as f:
            class_names_from_file = [line.strip() for line in f if line.strip()]
        for i, name in enumerate(class_names_from_file):
            label2id[name] = i
            id2label[i] = name
        print(f"Loaded classes: {id2label}")
    except FileNotFoundError:
        print("Error: 'classes.txt' not found. Please place it in the same directory as the script.")
        return
    except Exception as e:
        print(f"Error loading classes.txt: {e}")
        return

    # 3. 이미지 크기 확인 (학습 이미지 기준)
    image_files = glob.glob(os.path.join(train_images_path, "*.jpg")) + glob.glob(os.path.join(train_images_path, "*.png"))
    if not image_files:
        print(f"Error: No images found in {train_images_path}. Cannot proceed.")
        return

    first_image_path = image_files[0]
    img_temp = cv2.imread(first_image_path)
    if img_temp is None:
        print(f"Error: Could not read image {first_image_path}. Check file integrity.")
        return
    img_h, img_w = img_temp.shape[:2]
    print(f"Detected image dimensions from training data: W={img_w}, H={img_h}")

    # 4. Labelme JSON을 YOLO .txt로 변환 (Train & Val)
    print("\nConverting training labels (JSON -> YOLO .txt)...")
    convert_labelme_to_yolo_segmentation(train_labels_json_path, train_labels_yolo_path, img_w, img_h, label2id)
    print("Converting validation labels (JSON -> YOLO .txt)...")
    convert_labelme_to_yolo_segmentation(val_labels_json_path, val_labels_yolo_path, img_w, img_h, label2id)
    print("Label conversion complete.")

    # 5. data.yaml 파일 생성
    print("\nCreating 'data.yaml' configuration file...")
    yaml_path = os.path.join(dataset_path, "data.yaml")
    create_data_yaml(dataset_path, yaml_path, list(id2label.values()))

    # 6. YOLOv8 모델 학습
    print("\nLoading YOLOv8 model and starting training...")
    model = YOLO("yolov8n-seg.pt")

    try:
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_seg_{dataset_name}", # 결과 폴더 이름
            project=PROJECT_OUTPUT_PATH,
            save=True,      # 모델 저장
            resume=False    # 이어서 학습 안 함
        )
        print(f"--- Training for dataset '{dataset_name}' completed. ---")

        # 7. 검증 결과 출력 (학습 시 자동으로 검증이 수행됩니다)
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Validation Metrics ---")
            precision = results.metrics.get('metrics/precision(B)', 0.0)
            recall = results.metrics.get('metrics/recall(B)', 0.0)
            mAP50_B = results.metrics.get('metrics/mAP50(B)', 0.0)
            mAP_B = results.metrics.get('metrics/mAP50-95(B)', 0.0)
            mAP50_M = results.metrics.get('metrics/mAP50(M)', 0.0)
            mAP_M = results.metrics.get('metrics/mAP50-95(M)', 0.0)

            f1_score = 0.0
            if (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)

            print(f"Precision (Box): {precision:.4f}")
            print(f"Recall (Box): {recall:.4f}")
            print(f"F1-Score (Box): {f1_score:.4f}")
            print("-" * 20)
            print(f"mAP@.50 (Box): {mAP50_B:.4f}")
            print(f"mAP@.50-.95 (Box): {mAP_B:.4f}")
            print("-" * 20)
            print(f"mAP@.50 (Mask): {mAP50_M:.4f}")
            print(f"mAP@.50-.95 (Mask): {mAP_M:.4f}")
            print("------------------------")
        else:
            print("Could not retrieve validation metrics from training results.")

    except Exception as e:
        print(f"An error occurred during YOLO training for dataset '{dataset_name}': {e}")


# --- 스크립트 실행 ---
if __name__ == "__main__":
    # 학습을 진행할 데이터셋 폴더 이름을 지정합니다.
    # 예: BASE_DATASET_PATH가 "C:/.../augmented_data3"이고,
    # 그 안에 "my_dataset" 폴더가 있으며,
    # "my_dataset" 안에 "train", "val" 폴더가 있다면 "my_dataset"을 입력합니다.
    # 제공해주신 스크린샷 기준으로 폴더 이름이 없다면 새로 만드셔야 합니다.
    # 예를 들어, augmented_data3 안에 'dataset1' 폴더를 만들고 그 안에 train, val을 넣었다면
    # target_dataset = "dataset1" 로 설정합니다.
    target_dataset = "train" # ⚠️ 실제 데이터셋 폴더 이름으로 변경해주세요.

    # 에포크 수 설정
    num_epochs = 5

    run_yolo_training(target_dataset, epochs=num_epochs)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training for dataset: train ---
Loaded classes: {0: 'ac', 1: 'lc', 2: 'ph', 3: 'pc', 4: 'tc'}
Detected image dimensions from training data: W=1920, H=648

Converting training labels (JSON -> YOLO .txt)...
Converting validation labels (JSON -> YOLO .txt)...
Label conversion complete.

Creating 'data.yaml' configuration file...
'C:/Users/jenni/Desktop/augmented_data2\train\data.yaml' file created successfully.

Loading YOLOv8 model and starting training...
Ultralytics 8.3.152  Python-3.11.11 torch-2.7.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/jenni/Desktop/augmented_data2\train\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, e

train: Scanning C:\Users\jenni\Desktop\augmented_data2\train\train\labels... 1032 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1032/1032 [00:02<00:00, 495.47it/s]

train: New cache created: C:\Users\jenni\Desktop\augmented_data2\train\train\labels.cache



c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


val: Fast image access  (ping: 0.10.0 ms, read: 16.22.0 MB/s, size: 157.7 KB)


...

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  29%|██▊       | 2/7 [00:04<00:10,  2.15s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:13<00:00,  1.98s/it]
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\ultralytics\utils\metrics.py:585: RuntimeWarning: Mean of empty slice.
  ax.plot(px, py.mean(1), linewidth=3, color="blue", label=f"all classes {ap[:, 0].mean():.3f} mAP@0.5")
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\numpy\_core\_methods.py:144: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\ultralytics\utils\metrics.py:630: RuntimeWarning: Mean of empty slice.
  y = smooth(py.mean(0), 0.1)
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\numpy\_core\_methods.py:136: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\ultralytics\utils\metrics.py:630: RuntimeWarning: Mea

                   all        216          0          0          0          0          0          0          0          0          0
WARNING no labels found in segment set, can not compute metrics without labels
Speed: 0.7ms preprocess, 36.8ms inference, 0.0ms loss, 5.4ms postprocess per image
Results saved to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS2\yolov8n_seg_train
--- Training for dataset 'train' completed. ---
Could not retrieve validation metrics from training results.

All training processes finished.


In [1]:
#gpu 사용을 하도록

import os
import cv2
import json
import glob
from ultralytics import YOLO
import torch # Import torch to check for GPU availability


BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/augmented_data1" # 증강된 데이터의 최상위 경로 (poisson_harmonized 폴더가 이 아래에 있음)
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_TRAIN_RESULTS" # YOLO 학습 결과가 저장될 경로


def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """
    Labelme JSON 파일들을 YOLO segmentation .txt 파일로 변환합니다.
    Args:
        json_dir (str): Labelme JSON 파일들이 있는 디렉토리 경로.
        output_txt_dir (str): 변환된 YOLO .txt 파일들을 저장할 디렉토리 경로.
        img_w (int): 원본 이미지의 너비.
        img_h (int): 원본 이미지의 높이.
        label_to_id_map (dict): 클래스 이름(label)을 ID(숫자)로 매핑하는 딕셔너리.
    """
    os.makedirs(output_txt_dir, exist_ok=True)
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_filename = os.path.basename(json_path).replace(".json", ".txt")
        txt_path = os.path.join(output_txt_dir, txt_filename)

        yolo_lines = []
        for shape in data['shapes']:
            label_name = shape['label']
            if label_name in label_to_id_map:
                class_id = label_to_id_map[label_name]
                points = shape['points']
                
                normalized_points = []
                for x, y in points:
                    # 좌표를 이미지 크기에 맞춰 정규화
                    normalized_points.append(f"{(x / img_w):.6f}")
                    normalized_points.append(f"{(y / img_h):.6f}")
                
                yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
            else:
                print(f"Warning: Class '{label_name}' not found in label_to_id_map. Skipping shape in {json_path}")

        if yolo_lines:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(yolo_lines))
        # else:
        #    # 객체가 없는 이미지도 빈 .txt 파일로 저장하는 것이 YOLO 학습에 더 안전합니다.
        #    with open(txt_path, 'w') as f:
        #        pass # 빈 파일 생성


def create_data_yaml(dataset_base_path, yaml_output_path, class_names):
    """
    YOLOv8 학습을 위한 data.yaml 파일을 생성합니다.
    Args:
        dataset_base_path (str): 증강된 이미지와 라벨이 있는 상위 디렉토리 (예: 'C:/Users/USER/Desktop/augmented_data1/poisson_harmonized')
        yaml_output_path (str): 생성될 data.yaml 파일의 전체 경로.
        class_names (list): 데이터셋의 모든 클래스 이름 목록.
    """
    images_path = os.path.join(dataset_base_path, "images")
    labels_path = os.path.join(dataset_base_path, "labels_yolo") # YOLO 형식의 라벨 폴더

    # 경로를 상대 경로가 아닌 절대 경로로 지정하여 혼란 방지
    train_img_path = images_path
    val_img_path = images_path # 검증 데이터가 따로 없으면 train과 동일하게 설정

    nc = len(class_names)
    names = class_names

    yaml_content = f"""
path: {dataset_base_path}
train: {os.path.relpath(train_img_path, dataset_base_path)} # path 기준으로 상대 경로
val: {os.path.relpath(val_img_path, dataset_base_path)}

nc: {nc}
names: {names}
"""
    # 실제 YOLO 라벨 경로를 data.yaml이 참조하도록 labels 폴더의 이름을 변경하거나 (labels -> labels_yolo),
    # data.yaml 내부에서 라벨 경로를 명시적으로 지정해야 합니다.
    # 일반적으로 YOLO는 'images' 폴더와 같은 레벨의 'labels' 폴더를 기대합니다.
    # 그래서 'labels_yolo'라는 폴더명을 가정했습니다.

    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' file created successfully.")


def train_yolo_for_method(method, epochs=50):
    """
    특정 증강 방식에 대해 YOLOv8 모델을 학습시키고, 검증 결과를 도출합니다.
    Args:
        method (str): 증강 방식 이름 (예: 'poisson_harmonized').
        epochs (int): 학습할 에포크 수.
    """
    print(f"\n--- Starting YOLOv8 training and validation for method: {method} ---")

    # Check for GPU availability
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    if device == 'cpu':
        print("Warning: No GPU found. Training will be significantly slower on CPU. "
              "Ensure PyTorch with CUDA is installed correctly if you intend to use a GPU.")


    # 1. 각 증강 방식에 맞는 데이터셋 경로 설정
    dataset_path = os.path.join(BASE_AUGMENTED_PATH, method)
    images_path = os.path.join(dataset_path, "images")
    labels_json_path = os.path.join(dataset_path, "labels") # Labelme JSON 라벨이 있는 곳
    labels_yolo_path = os.path.join(dataset_path, "labels_yolo") # YOLO .txt 라벨을 저장할 곳

    # 2. 클래스 정보를 미리 정의
    label2id = {}
    id2label = {}
    try:
        with open("classes.txt", "r", encoding="utf-8") as f:
            class_names_from_file = [line.strip() for line in f if line.strip()]
        for i, name in enumerate(class_names_from_file):
            label2id[name] = i
            id2label[i] = name
        print(f"Loaded classes: {id2label}")
    except FileNotFoundError:
        print("Error: 'classes.txt' not found. Please ensure it's in the same directory or provide class info manually.")
        return
    except Exception as e:
        print(f"Error loading classes.txt: {e}")
        return

    # 이미지 크기를 추정하기 위해 첫 번째 이미지 파일을 읽습니다.
    first_image_path = None
    image_files = glob.glob(os.path.join(images_path, "*.jpg")) + glob.glob(os.path.join(images_path, "*.png"))
    if image_files:
        first_image_path = image_files[0]
        img_temp = cv2.imread(first_image_path)
        if img_temp is None:
            print(f"Error: Could not read image {first_image_path}. Check image file integrity.")
            return
        img_h, img_w = img_temp.shape[:2]
        print(f"Detected image dimensions: W={img_w}, H={img_h}")
    else:
        print(f"Error: No images found in {images_path}. Cannot determine image dimensions for YOLO conversion.")
        return

    # 3. Labelme JSON 라벨을 YOLO Segmentation .txt 형식으로 변환
    print(f"Converting Labelme JSON to YOLO segmentation .txt for {method}...")
    convert_labelme_to_yolo_segmentation(labels_json_path, labels_yolo_path, img_w, img_h, label2id)
    print("Conversion complete.")

    # 4. data.yaml 파일 생성
    yaml_path = os.path.join(dataset_path, "data.yaml")
    create_data_yaml(dataset_path, yaml_path, list(id2label.values()))

    # 5. 항상 동일한 사전 학습 모델로부터 학습 시작
    model = YOLO("yolov8n-seg.pt")
    print(f"Loaded YOLOv8n-seg model.")

    # 6. 모든 학습에 동일한 하이퍼파라미터 적용
    try:
        # 학습 수행
        results = model.train(
            data=yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{method}",
            project=PROJECT_OUTPUT_PATH, # 결과 저장 최상위 경로
            save=True,
            resume=False,
            device=device # Explicitly set device
        )
        print(f"--- Training for method '{method}' completed. Results saved to {PROJECT_OUTPUT_PATH}/yolov8n_{method} ---")

        # 7. 학습 완료 후 검증 수행 (자동으로 수행되지만 명시적으로 호출 가능)
        if hasattr(results, 'metrics') and results.metrics:
            print("\n--- Validation Metrics ---")
            
            precision = results.metrics.get('metrics/precision(B)', 'N/A')
            recall = results.metrics.get('metrics/recall(B)', 'N/A')
            mAP50 = results.metrics.get('metrics/mAP50(B)', 'N/A')
            mAP = results.metrics.get('metrics/mAP50-95(B)', 'N/A') # mAP (0.5 to 0.95 IOU)

            # F1-score 계산
            f1_score = 'N/A'
            if precision != 'N/A' and recall != 'N/A' and (precision + recall) > 0:
                f1_score = 2 * (precision * recall) / (precision + recall)

            print(f"Precision: {precision:.4f}")
            print(f"Recall: {recall:.4f}")
            print(f"F1-Score: {f1_score:.4f}")
            print(f"mAP50: {mAP50:.4f}")
            print(f"mAP (50-95): {mAP:.4f}")
            print("------------------------")
        else:
            print("Could not retrieve validation metrics from training results.")

    except Exception as e:
        print(f"Error during YOLO training for method '{method}': {e}")


# --- 사용 예시 ---
if __name__ == "__main__":
    # 학습을 시작할 증강 방식 이름 지정 (예: 이전 증강 스크립트에서 사용한 blending mode와 동일하게)
    target_method = "train" # ⚠️ 여기에 실제로 사용한 증강 방식 폴더 이름을 입력하세요.

    train_yolo_for_method(target_method, epochs=50)

    print("\nAll training processes finished.")


--- Starting YOLOv8 training and validation for method: train ---
Using device: cpu
Loaded classes: {0: 'ac', 1: 'lc', 2: 'ph', 3: 'pc', 4: 'tc'}
Detected image dimensions: W=1920, H=648
Converting Labelme JSON to YOLO segmentation .txt for train...
Conversion complete.
'C:/Users/jenni/Desktop/augmented_data1\train\data.yaml' file created successfully.
Loaded YOLOv8n-seg model.
Ultralytics 8.3.152  Python-3.11.11 torch-2.7.1+cpu CPU (11th Gen Intel Core(TM) i7-11800H 2.30GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/jenni/Desktop/augmented_data1\train\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, f

train: Scanning C:\Users\jenni\Desktop\augmented_data1\train\labels.cache... 991 images, 9 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.00.0 ms, read: 1755.1391.6 MB/s, size: 323.6 KB)



c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\jenni\Desktop\augmented_data1\train\labels.cache... 991 images, 9 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:00<?, ?it/s]
c:\Users\jenni\anaconda3\envs\aug\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Plotting labels to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS\yolov8n_train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS\yolov8n_train2
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   6%|▋         | 2/32 [00:04<01:01,  2.05s/it]

WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validation plots to first 50 items per image for speed...
WARNING Limiting validati

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 32/32 [00:52<00:00,  1.64s/it]


                   all       1000       2931      0.608      0.369      0.407      0.244      0.558      0.286      0.304      0.129
                    ac         48         50      0.616      0.579      0.605      0.405      0.464       0.38      0.405      0.197
                    lc        103        109      0.495      0.339      0.358      0.218      0.367       0.22      0.197     0.0605
                    ph        730       1202      0.717      0.453      0.502      0.303      0.738      0.424       0.48      0.222
                    pc        406        524      0.609      0.139      0.177      0.099      0.639      0.134      0.168     0.0752
                    tc        654       1046      0.603      0.337      0.391      0.196      0.581      0.273      0.269       0.09
Speed: 0.7ms preprocess, 30.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to C:\Users\jenni\Desktop\YOLO_TRAIN_RESULTS\yolov8n_train2
--- Training for method 'train' completed. Re

In [12]:
#val 데이터를 활용하여 지표 검증증
import os
import cv2
import json
import glob
from ultralytics import YOLO
import torch # Import torch to check for GPU availability

# --- Paths ---
BASE_AUGMENTED_PATH = "C:/Users/jenni/Desktop/augmented_data1" # 증강된 데이터의 최상위 경로 (train 폴더가 이 아래에 있음)
# Assuming your original dataset has a 'val' split that you want to use for validation
VAL_DATA_BASE_PATH = "C:/Users/jenni/Desktop/original_data1" # ⚠️ 여기에 실제 원본 데이터셋의 최상위 경로를 입력하세요.
                                                              # 이 경로 아래에 'val_images'와 'val_labels' 폴더가 있다고 가정합니다.
PROJECT_OUTPUT_PATH = "C:/Users/jenni/Desktop/YOLO_TRAIN_RESULTS_with_val" # YOLO 학습 결과가 저장될 경로

def convert_labelme_to_yolo_segmentation(json_dir, output_txt_dir, img_w, img_h, label_to_id_map):
    """
    Labelme JSON 파일들을 YOLO segmentation .txt 파일로 변환합니다.
    Args:
        json_dir (str): Labelme JSON 파일들이 있는 디렉토리 경로.
        output_txt_dir (str): 변환된 YOLO .txt 파일들을 저장할 디렉토리 경로.
        img_w (int): 원본 이미지의 너비.
        img_h (int): 원본 이미지의 높이.
        label_to_id_map (dict): 클래스 이름(label)을 ID(숫자)로 매핑하는 딕셔너리.
    """
    os.makedirs(output_txt_dir, exist_ok=True)
    json_files = glob.glob(os.path.join(json_dir, "*.json"))

    if not json_files:
        print(f"Warning: No JSON files found in {json_dir}. No YOLO labels will be generated.")
        return

    for json_path in json_files:
        with open(json_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        txt_filename = os.path.basename(json_path).replace(".json", ".txt")
        txt_path = os.path.join(output_txt_dir, txt_filename)

        yolo_lines = []
        # Ensure image dimensions are valid before normalization
        if img_w <= 0 or img_h <= 0:
            print(f"Error: Invalid image dimensions (W={img_w}, H={img_h}) detected for {json_path}. Skipping.")
            continue

        for shape in data['shapes']:
            label_name = shape['label']
            if label_name in label_to_id_map:
                class_id = label_to_id_map[label_name]
                points = shape['points']
                
                normalized_points = []
                for x, y in points:
                    # Clip coordinates to be within image bounds before normalization
                    x = max(0.0, min(float(img_w), float(x)))
                    y = max(0.0, min(float(img_h), float(y)))
                    
                    # Normalize coordinates
                    normalized_points.append(f"{(x / img_w):.6f}")
                    normalized_points.append(f"{(y / img_h):.6f}")
                
                yolo_lines.append(f"{class_id} {' '.join(normalized_points)}")
            else:
                print(f"Warning: Class '{label_name}' not found in label_to_id_map. Skipping shape in {json_path}")

        if yolo_lines:
            with open(txt_path, 'w', encoding='utf-8') as f:
                f.write('\n'.join(yolo_lines))
        else:
            # If no objects are found or processed, create an empty .txt file
            # This is important for YOLO to understand that the image has no objects.
            with open(txt_path, 'w') as f:
                pass


def create_data_yaml(yaml_output_path, train_img_dir, val_img_dir, train_label_dir, val_label_dir, class_names):
    """
    YOLOv8 학습 및 검증을 위한 data.yaml 파일을 생성합니다.
    Args:
        yaml_output_path (str): 생성될 data.yaml 파일의 전체 경로.
        train_img_dir (str): 학습 이미지 디렉토리의 절대 경로.
        val_img_dir (str): 검증 이미지 디렉토리의 절대 경로.
        train_label_dir (str): 학습 YOLO .txt 라벨 디렉토리의 절대 경로.
        val_label_dir (str): 검증 YOLO .txt 라벨 디렉토리의 절대 경로.
        class_names (list): 데이터셋의 모든 클래스 이름 목록.
    """
    nc = len(class_names)
    names = class_names

    # YOLOv8는 train/val/test 이미지가 하나의 'path' 아래에 상대 경로로 있거나,
    # 각 train/val/test 경로를 절대 경로로 직접 지정할 수 있습니다.
    # 여기서는 절대 경로를 사용하여 명확하게 지정합니다.
    yaml_content = f"""
# Dataset root directory (not strictly needed if train/val/test paths are absolute)
# path: {os.path.dirname(train_img_dir)} 

train:
  images: {train_img_dir}
  labels: {train_label_dir}
val:
  images: {val_img_dir}
  labels: {val_label_dir}

nc: {nc}
names: {names}
"""
    with open(yaml_output_path, "w", encoding="utf-8") as f:
        f.write(yaml_content.strip())
    print(f"'{yaml_output_path}' file created successfully.")


def get_image_dimensions(image_dir):
    """
    주어진 디렉토리의 첫 번째 이미지 파일에서 이미지 크기를 가져옵니다.
    """
    image_files = glob.glob(os.path.join(image_dir, "*.jpg")) + glob.glob(os.path.join(image_dir, "*.png"))
    if image_files:
        first_image_path = image_files[0]
        img_temp = cv2.imread(first_image_path)
        if img_temp is None:
            print(f"Error: Could not read image {first_image_path}. Check image file integrity.")
            return None, None
        return img_temp.shape[1], img_temp.shape[0] # width, height
    else:
        print(f"Error: No images found in {image_dir}.")
        return None, None

def load_class_info(class_file="classes.txt"):
    """
    'classes.txt' 파일에서 클래스 이름과 ID 매핑을 로드합니다.
    """
    label2id = {}
    id2label = {}
    try:
        with open(class_file, "r", encoding="utf-8") as f:
            class_names_from_file = [line.strip() for line in f if line.strip()]
        for i, name in enumerate(class_names_from_file):
            label2id[name] = i
            id2label[i] = name
        print(f"Loaded classes: {id2label}")
        return label2id, id2label, list(id2label.values())
    except FileNotFoundError:
        print(f"Error: '{class_file}' not found. Please ensure it's in the same directory or provide class info manually.")
        return {}, {}, []
    except Exception as e:
        print(f"Error loading {class_file}: {e}")
        return {}, {}, []


def train_yolo_model(train_method, epochs=50):
    """
    특정 증강 방식에 대해 YOLOv8 모델을 학습시킵니다.
    학습 후, 저장된 모델 경로를 반환합니다.
    Args:
        train_method (str): 증강 방식 이름 (예: 'train').
        epochs (int): 학습할 에포크 수.
    Returns:
        str: 학습된 모델이 저장될 경로 (예: 'runs/segment/yolov8n_train/weights/best.pt')
    """
    print(f"\n--- Starting YOLOv8 training for method: {train_method} ---")

    # Check for GPU availability
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"Using device: {device}")
    if device == 'cpu':
        print("Warning: No GPU found. Training will be significantly slower on CPU. "
              "Ensure PyTorch with CUDA is installed correctly if you intend to use a GPU.")

    # 1. 학습 데이터셋 경로 설정 (증강된 데이터)
    train_dataset_base_path = os.path.join(BASE_AUGMENTED_PATH, train_method)
    train_images_path = os.path.join(train_dataset_base_path, "images")
    train_labels_json_path = os.path.join(train_dataset_base_path, "labels") # Labelme JSON 라벨이 있는 곳
    train_labels_yolo_path = os.path.join(train_dataset_base_path, "labels_yolo") # YOLO .txt 라벨을 저장할 곳

    # 2. 검증 데이터셋 경로 설정 (원본 데이터셋의 val 폴더)
    val_images_path = os.path.join(VAL_DATA_BASE_PATH, "val_images")
    val_labels_json_path = os.path.join(VAL_DATA_BASE_PATH, "val_labels")
    val_labels_yolo_path = os.path.join(VAL_DATA_BASE_PATH, "val_labels_yolo")

    # 3. 클래스 정보 로드
    label2id, id2label, class_names = load_class_info()
    if not class_names: return None # Exit if classes are not loaded

    # 4. 학습 및 검증 이미지 크기 추정 (YOLO 변환에 필요)
    train_img_w, train_img_h = get_image_dimensions(train_images_path)
    val_img_w, val_img_h = get_image_dimensions(val_images_path)

    if train_img_w is None or val_img_w is None:
        print("Error: Could not determine image dimensions for training or validation. Exiting.")
        return None

    # 5. Labelme JSON 라벨을 YOLO Segmentation .txt 형식으로 변환 (학습 및 검증 데이터셋 모두)
    print(f"Converting Labelme JSON to YOLO segmentation .txt for training data ({train_method})...")
    convert_labelme_to_yolo_segmentation(train_labels_json_path, train_labels_yolo_path, train_img_w, train_img_h, label2id)
    print("Training data conversion complete.")

    print(f"Converting Labelme JSON to YOLO segmentation .txt for validation data...")
    convert_labelme_to_yolo_segmentation(val_labels_json_path, val_labels_yolo_path, val_img_w, val_img_h, label2id)
    print("Validation data conversion complete.")

    # 6. data.yaml 파일 생성 (학습과 검증 경로를 모두 포함)
    data_yaml_path = os.path.join(PROJECT_OUTPUT_PATH, f"{train_method}_data.yaml")
    create_data_yaml(
        yaml_output_path=data_yaml_path,
        train_img_dir=train_images_path,
        val_img_dir=val_images_path,
        train_label_dir=train_labels_yolo_path,
        val_label_dir=val_labels_yolo_path,
        class_names=class_names
    )

    # 7. 항상 동일한 사전 학습 모델로부터 학습 시작
    model = YOLO("yolov8n-seg.pt")
    print(f"Loaded YOLOv8n-seg model.")

    # 8. 학습 수행
    try:
        # YOLOv8 train function automatically performs validation if 'val' path is given in data.yaml
        results = model.train(
            data=data_yaml_path,
            epochs=epochs,
            imgsz=640,
            name=f"yolov8n_{train_method}",
            project=PROJECT_OUTPUT_PATH, # 결과 저장 최상위 경로
            save=True,
            resume=False,
            device=device
        )
        print(f"--- Training for method '{train_method}' completed. ---")

        # Get the path to the best trained model
        # The 'path' attribute of the returned results object points to the run directory
        # The 'best.pt' model is usually saved within this directory.
        run_dir = os.path.join(PROJECT_OUTPUT_PATH, f"yolov8n_{train_method}")
        best_model_path = os.path.join(run_dir, "weights", "best.pt")
        if os.path.exists(best_model_path):
            print(f"Best model saved to: {best_model_path}")
            return best_model_path
        else:
            print(f"Warning: 'best.pt' not found at {best_model_path}. Check training logs for exact path.")
            # Fallback to last.pt if best.pt is not found (though best.pt is preferred)
            last_model_path = os.path.join(run_dir, "weights", "last.pt")
            if os.path.exists(last_model_path):
                print(f"Using 'last.pt' model at: {last_model_path}")
                return last_model_path
            else:
                print(f"Error: Neither 'best.pt' nor 'last.pt' found in {run_dir}.")
                return None

    except Exception as e:
        print(f"Error during YOLO training for method '{train_method}': {e}")
        return None

def validate_yolo_model(trained_model_path, validation_data_yaml_path):
    """
    학습된 YOLOv8 모델을 사용하여 별도의 검증 데이터셋에 대해 성능을 평가합니다.
    Args:
        trained_model_path (str): 학습된 모델 파일의 경로 (예: 'runs/segment/yolov8n_train/weights/best.pt').
        validation_data_yaml_path (str): 검증 데이터셋에 대한 data.yaml 파일 경로.
    """
    if not trained_model_path or not os.path.exists(trained_model_path):
        print(f"Error: Trained model not found at {trained_model_path}. Cannot perform validation.")
        return

    if not os.path.exists(validation_data_yaml_path):
        print(f"Error: Validation data YAML not found at {validation_data_yaml_path}. Cannot perform validation.")
        return

    print(f"\n--- Starting YOLOv8 validation for model: {os.path.basename(trained_model_path)} ---")
    print(f"Using validation data specified in: {validation_data_yaml_path}")

    # Load the trained model
    model = YOLO(trained_model_path)
    print(f"Loaded trained model: {trained_model_path}")

    # Perform validation
    try:
        metrics = model.val(data=validation_data_yaml_path)

        print("\n--- Validation Metrics ---")
        precision = metrics.box.p # Precision for bounding box detection
        recall = metrics.box.r    # Recall for bounding box detection
        mAP50 = metrics.box.map50 # mAP@0.5 for bounding box detection
        mAP = metrics.box.map     # mAP@0.5-0.95 for bounding box detection

        # Segmentation metrics (often denoted with seg in Ultralytics)
        # Note: If your model is a segmentation model (e.g., yolov8n-seg), these will be available.
        # If it's a detection model, these might be 0 or not present.
        seg_mAP50 = getattr(metrics.seg, 'map50', 'N/A')
        seg_mAP = getattr(metrics.seg, 'map', 'N/A')

        # F1-score calculation (for bounding box detection)
        f1_score = 'N/A'
        if precision is not None and recall is not None and (precision + recall) > 0:
            f1_score = 2 * (precision * recall) / (precision + recall)
        
        print(f"Box Precision: {precision:.4f}")
        print(f"Box Recall: {recall:.4f}")
        print(f"Box F1-Score: {f1_score:.4f}")
        print(f"Box mAP50: {mAP50:.4f}")
        print(f"Box mAP (50-95): {mAP:.4f}")
        
        if seg_mAP50 != 'N/A': # Check if segmentation metrics are available
            print(f"Seg mAP50: {seg_mAP50:.4f}")
            print(f"Seg mAP (50-95): {seg_mAP:.4f}")
        else:
            print("Segmentation metrics not available (model might not be segmentation, or no masks found).")
            
        print("------------------------")

    except Exception as e:
        print(f"Error during YOLO validation: {e}")

# --- Main execution ---
if __name__ == "__main__":
    # 학습을 시작할 증강 방식 이름 지정 (예: 이전 증강 스크립트에서 사용한 blending mode와 동일하게)
    # BASE_AUGMENTED_PATH/train 폴더에 학습 데이터가 있다고 가정
    train_method_name = "train" 

    # 1. Train the model using the augmented data
    trained_model_path = train_yolo_model(train_method_name, epochs=50)

    # 2. After training, validate the model using the separate validation set
    if trained_model_path:
        # Create a data.yaml specifically for validation, pointing to the validation data
        # We need to recreate it because the 'train' method's data.yaml might be in the run directory
        # and we want to ensure the validation step explicitly uses the correct validation paths.
        
        # Reload class info to ensure consistency
        label2id, id2label, class_names = load_class_info()
        if not class_names:
            print("Failed to load class information. Cannot proceed with validation.")
        else:
            val_images_path = os.path.join(VAL_DATA_BASE_PATH, "val_images")
            val_labels_yolo_path = os.path.join(VAL_DATA_BASE_PATH, "val_labels_yolo")

            # Validate images and labels exist
            if not os.path.exists(val_images_path) or not os.path.exists(val_labels_yolo_path):
                print(f"Error: Validation images or labels path missing. Please check '{val_images_path}' and '{val_labels_yolo_path}'.")
            else:
                # Get the image dimensions for the validation set, as YOLO conversion needs it.
                # Assuming all images in val_images have the same size.
                val_img_w, val_img_h = get_image_dimensions(val_images_path)
                if val_img_w is None:
                    print("Could not get dimensions for validation images. Skipping validation.")
                else:
                    # Ensure validation labels are converted if they weren't already (e.g., if you run this script directly)
                    # This step might be redundant if train_yolo_model already converted them, but it's safe.
                    val_labels_json_path = os.path.join(VAL_DATA_BASE_PATH, "val_labels")
                    convert_labelme_to_yolo_segmentation(val_labels_json_path, val_labels_yolo_path, val_img_w, val_img_h, label2id)
                    
                    # Create a data.yaml for validation specifically
                    validation_data_yaml_path = os.path.join(PROJECT_OUTPUT_PATH, f"{train_method}_val_data.yaml")
                    create_data_yaml(
                        yaml_output_path=validation_data_yaml_path,
                        train_img_dir=os.path.join(BASE_AUGMENTED_PATH, train_method, "images"), # Placeholder, not used by val
                        val_img_dir=val_images_path,
                        train_label_dir=os.path.join(BASE_AUGMENTED_PATH, train_method, "labels_yolo"), # Placeholder
                        val_label_dir=val_labels_yolo_path,
                        class_names=class_names
                    )
                    validate_yolo_model(trained_model_path, validation_data_yaml_path)
    else:
        print("Training did not complete successfully, skipping validation.")

    print("\nAll processes finished.")


--- Starting YOLOv8 training for method: train ---
Using device: cpu
Loaded classes: {0: 'ac', 1: 'lc', 2: 'ph', 3: 'pc', 4: 'tc'}
Converting Labelme JSON to YOLO segmentation .txt for training data (train)...
Training data conversion complete.
Converting Labelme JSON to YOLO segmentation .txt for validation data...
Validation data conversion complete.


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/jenni/Desktop/YOLO_TRAIN_RESULTS_with_val\\train_data.yaml'

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install ultralytics

In [1]:
#json 파일일 떄 
import os
import shutil
import random
import glob

# 원본 train 폴더 경로 (images와 labels 폴더가 이 안에 있다고 가정)
SOURCE_TRAIN_DIR = "C:/Users/USER/Desktop/augmented1/train"

# 추출된 파일들을 저장할 새 폴더 경로
DESTINATION_DIR = "C:/Users/USER/Desktop/train_100_samples"

# 추출할 파일 개수
NUM_SAMPLES = 100

def extract_samples(source_base_dir, dest_base_dir, num_samples=100):
    """
    원본 폴더에서 이미지와 해당 라벨 파일을 num_samples 개수만큼 추출하여 새 폴더로 복사합니다.
    """
    source_images_dir = os.path.join(source_base_dir, "images")
    source_labels_dir = os.path.join(source_base_dir, "labels") # Labelme JSON 라벨 폴더
    source_labels_yolo_dir = os.path.join(source_base_dir, "labels_yolo") # YOLO .txt 라벨 폴더

    dest_images_dir = os.path.join(dest_base_dir, "images")
    dest_labels_dir = os.path.join(dest_base_dir, "labels")
    dest_labels_yolo_dir = os.path.join(dest_base_dir, "labels_yolo")


    # 대상 폴더 생성
    os.makedirs(dest_images_dir, exist_ok=True)
    os.makedirs(dest_labels_dir, exist_ok=True)
    os.makedirs(dest_labels_yolo_dir, exist_ok=True)


    # 이미지 파일 목록 가져오기 (jpg, png 모두 포함)
    image_files = []
    image_files.extend(glob.glob(os.path.join(source_images_dir, "*.jpg")))
    image_files.extend(glob.glob(os.path.join(source_images_dir, "*.png")))

    if not image_files:
        print(f"Error: No image files found in {source_images_dir}")
        return

    print(f"Found {len(image_files)} image files.")

    # 충분한 파일이 있는지 확인
    if len(image_files) < num_samples:
        print(f"Warning: Only {len(image_files)} image files found, less than requested {num_samples}. Copying all found files.")
        selected_images = image_files
    else:
        # 무작위로 num_samples 개수만큼 이미지 선택
        selected_images = random.sample(image_files, num_samples)
        # 또는 순서대로 선택하고 싶다면: selected_images = image_files[:num_samples]

    print(f"Selected {len(selected_images)} image files for sampling.")

    copied_count = 0
    for img_path in selected_images:
        img_filename = os.path.basename(img_path)
        base_filename = os.path.splitext(img_filename)[0]

        # 이미지 복사
        dest_img_path = os.path.join(dest_images_dir, img_filename)
        shutil.copy(img_path, dest_img_path)

        # 해당 Labelme JSON 라벨 파일 복사
        json_label_path = os.path.join(source_labels_dir, f"{base_filename}.json")
        if os.path.exists(json_label_path):
            dest_json_label_path = os.path.join(dest_labels_dir, f"{base_filename}.json")
            shutil.copy(json_label_path, dest_json_label_path)
        else:
            print(f"Warning: Labelme JSON file not found for {img_filename} at {json_label_path}. Skipping.")

        # 해당 YOLO .txt 라벨 파일 복사
        yolo_label_path = os.path.join(source_labels_yolo_dir, f"{base_filename}.txt")
        if os.path.exists(yolo_label_path):
            dest_yolo_label_path = os.path.join(dest_labels_yolo_dir, f"{base_filename}.txt")
            shutil.copy(yolo_label_path, dest_yolo_label_path)
        else:
            print(f"Warning: YOLO .txt file not found for {img_filename} at {yolo_label_path}. Skipping.")
        
        copied_count += 1

    print(f"\n--- Successfully copied {copied_count} image-label pairs. ---")
    print(f"Samples are saved in: {DESTINATION_DIR}")


if __name__ == "__main__":
    extract_samples(SOURCE_TRAIN_DIR, DESTINATION_DIR, NUM_SAMPLES)

Found 2350 image files.
Selected 100 image files for sampling.

--- Successfully copied 100 image-label pairs. ---
Samples are saved in: C:/Users/USER/Desktop/train_100_samples
